In [1]:
# import libraries
import requests
import json
from dataclasses import dataclass, asdict
from typing import List
import pandas as pd

In [2]:
# Define models
@dataclass
class Order:
    symbol: str
    qty: int
    side: str
    type: str
    price: float
    time_in_force: str
        
@dataclass
class Trade:
    symbol: str
    qty: int
    price: float
    side: str
    timestamp: str
    order_id: str
    instmt: str
    trade_price: str
    trade_qty: str
    trade_side: str
    trade_id: int
    @classmethod
    def from_dict(cls, data: dict) -> 'Trade':
        return cls(**data)
    
@dataclass
class StockDetails:
    symbol: str
    name: str
    exchange: str
    currency: str
    country: str
    sector: str
    industry: str
    market_cap: float
    beta: float
    pe_ratio: float
    eps: float
    dividend_yield: float
    dividend_per_share: float
    dividend_pay_date: str
    dividend_ex_date: str
    last_split_factor: str
    last_split_date: str    
    @classmethod
    def from_dict(cls, data: dict) -> 'StockDetails':
        return cls(**data)
      
@dataclass
class TopStock:
    symbol: str
    name: str
    change_pct: float
    price: float

In [8]:
# Define the base URL of the dummy stock exchange API
base_url = 'http://localhost:5000/api/'

# services
def get_stock_detail(symbol)-> StockDetails:
    endpoint = 'v1/stock/'+symbol
    response = requests.get(base_url+ endpoint)
    if response.status_code == 200:
        stock_dict = response.json()
        stock = StockDetails.from_dict(stock_dict)
        return stock
    else:
        raise Exception("Internal server error")
    
def place_order(order) -> Trade:
    endpoint = 'v1/orders/place'
    response = requests.post(base_url + endpoint, json=asdict(order))

    if response.status_code == 200:
        print("Order Placed Successfully")
        respdict = response.json()
        trade = Trade.from_dict(respdict)
        return trade
    else:
        raise Exception("Order was not executed")
        
def cancel_order(order):
    # Define the endpoint for placing an order
    endpoint = 'v1/orders/cancel'
    # Send the request to place the order
    response = requests.delete(base_url + endpoint, json=asdict(order))
    # Print the response
    print(response.status_code)
    
    # Print the trade response
    if response.status_code == 200:
        trade_dict = response.json()['trade']
        print("Order deleted")
    else:
        raise Exception("Order was not cancelled")

In [9]:
tradersTop10Stocks = ['AAPL','MSFT','GOOG','AMZN','PCAR','TSLA','META']

In [10]:
# Program should first read the list of instruments (with relevant details) available on the exchange. 
stocks = [get_stock_detail(tradersTop10Stocks[0])]

In [11]:
from prettytable import PrettyTable

# Assuming you have a list of StockDetails objects called 'stocks'
table = PrettyTable()
table.field_names = ["Symbol", "Name", "Exchange", "Currency", "Country", "Sector", "Industry", "Market Cap", "Beta", "P/E Ratio", "EPS", "Dividend Yield", "Dividend/Share", "Dividend Pay Date", "Dividend Ex Date", "Last Split Factor", "Last Split Date"]

for stock in stocks:
    table.add_row([stock.symbol, stock.name, stock.exchange, stock.currency, stock.country, stock.sector, stock.industry, stock.market_cap, stock.beta, stock.pe_ratio, stock.eps, stock.dividend_yield, stock.dividend_per_share, stock.dividend_pay_date, stock.dividend_ex_date, stock.last_split_factor, stock.last_split_date])

print(table)


+--------+------------+----------+----------+---------+------------+----------------------+------------+------+-----------+------+----------------+----------------+-------------------+------------------+-------------------+-----------------+
| Symbol |    Name    | Exchange | Currency | Country |   Sector   |       Industry       | Market Cap | Beta | P/E Ratio | EPS  | Dividend Yield | Dividend/Share | Dividend Pay Date | Dividend Ex Date | Last Split Factor | Last Split Date |
+--------+------------+----------+----------+---------+------------+----------------------+------------+------+-----------+------+----------------+----------------+-------------------+------------------+-------------------+-----------------+
|  AAPL  | Apple Inc. |  NASDAQ  |   USD    |   USA   | Technology | Consumer Electronics | 2234926.77 | 1.22 |   30.92   | 5.19 |      0.61      |      0.88      |     2021-11-11    |    2021-11-04    |        7:1        |    2014-06-09   |
+--------+------------+---------

In [12]:
order = Order(symbol='AAPL', qty=100, side='buy', type='limit', price=150.0, time_in_force='gtc')
trade = place_order(order)

Order Placed Successfully


In [13]:
# Assuming you have a list of StockDetails objects called 'stocks'
table = PrettyTable()
table.field_names = ["Symbol", "Quantity", "Price", "Side", "OrderPlacedAt",
                     "OrderId", "Stock","Trade Price","Trade Quantity","Trade Side","TradeId"]
table.add_row([trade.symbol, trade.qty, trade.price, trade.side, trade.timestamp, trade.order_id, trade.instmt,
               trade.trade_price,  trade.trade_qty,trade.trade_side,trade.trade_id])
print(table)

+--------+----------+-------+------+----------------------------+--------------------------------------+-------+-------------+----------------+------------+--------------------------------------+
| Symbol | Quantity | Price | Side |       OrderPlacedAt        |               OrderId                | Stock | Trade Price | Trade Quantity | Trade Side |               TradeId                |
+--------+----------+-------+------+----------------------------+--------------------------------------+-------+-------------+----------------+------------+--------------------------------------+
|  AAPL  |   100    | 150.0 | buy  | 2023-03-11T06:22:45.228128 | 6d78a5fc-e037-4e15-a3d3-7b13fb9592ef |  AAPL |    150.0    |      100       |    buy     | 5150f1ca-46a0-484d-a316-1826127beeab |
+--------+----------+-------+------+----------------------------+--------------------------------------+-------+-------------+----------------+------------+--------------------------------------+


<h3>Generate orders in bulk</h3>

In [26]:
import random
import datetime

# Define a list of stocks and their historical prices
stock_prices = {
    'AAPL': [150, 152, 151, 149, 153, 155, 156, 157, 159, 160],
    'GOOG': [1200, 1210, 1220, 1205, 1195, 1185, 1190, 1180, 1175, 1165],
    'TSLA': [700, 710, 720, 730, 735, 740, 745, 750, 755, 760],
    'AMZN': [3000, 3010, 3020, 3015, 3012, 3014, 3018, 3016, 3013, 3011],
    'FB': [350, 355, 360, 365, 370, 375, 380, 385, 390, 395]
}

# Define a list of possible sides
sides = ['BUY', 'SELL']

# Define a list of possible quantities
quantities = list(range(1, 11))

# Define a function to generate a random order
def generate_order():
    # Generate a random stock symbol from the list
    stock = random.choice(list(stock_prices.keys()))

    # Get the last price of the stock
    last_price = stock_prices[stock][-1]

    # Generate a random quantity between 1 and 10
    quantity = random.choice(quantities)

    # Generate a random buy or sell side
    side = random.choice(sides)

    # Generate a realistic price based on the last price of the stock
    if side == 'BUY':
        # Add a random percentage to the last price
        price = round(last_price * (1 + random.uniform(0, 0.05)), 2)
    else:
        # Subtract a random percentage from the last price
        price = round(last_price * (1 - random.uniform(0, 0.05)), 2)

    # Get the current time
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Create the order dictionary using the randomly generated values
    order = Order(symbol=f'{stock}', qty=quantity, side=f'{side}', type='limit', price=price, time_in_force=timestamp)  
    return order

# Generate 100 random orders
for i in range(100):
    # Generate a random order
    order = generate_order()
    # Print the order statement
    print(f"Randomly generated order statement {i+1}: {order}")    
    trade = place_order(order)
    print(trade)

Randomly generated order statement 1: Order(symbol='FB', qty=5, side='BUY', type='limit', price=395.48, time_in_force='2023-03-11 14:10:22')
Order Placed Successfully
Trade(symbol='FB', qty=5, price=395.48, side='BUY', timestamp='2023-03-11T19:10:22.614726', order_id='b90c4081-4b8c-4659-9e68-d24fe9c8d434', instmt='FB', trade_price=395.48, trade_qty=5, trade_side='BUY', trade_id='6e6e608a-7de9-4fdc-9262-d188d71afd20')
Randomly generated order statement 2: Order(symbol='GOOG', qty=6, side='BUY', type='limit', price=1215.95, time_in_force='2023-03-11 14:10:22')
Order Placed Successfully
Trade(symbol='GOOG', qty=6, price=1215.95, side='BUY', timestamp='2023-03-11T19:10:22.621930', order_id='9be520e5-663e-4f33-a8c0-bbe05b11cdf5', instmt='GOOG', trade_price=1215.95, trade_qty=6, trade_side='BUY', trade_id='3fc61fb1-20ae-470d-9d31-d798b60cf72a')
Randomly generated order statement 3: Order(symbol='TSLA', qty=9, side='SELL', type='limit', price=753.77, time_in_force='2023-03-11 14:10:22')
Order

Order Placed Successfully
Trade(symbol='AAPL', qty=2, price=155.65, side='SELL', timestamp='2023-03-11T19:10:22.810985', order_id='a52ca920-b9f8-4a7e-8d08-7b0b1be9cb1f', instmt='AAPL', trade_price=155.65, trade_qty=2, trade_side='SELL', trade_id='092027c9-c249-4b63-8df2-df063f470678')
Randomly generated order statement 43: Order(symbol='FB', qty=2, side='BUY', type='limit', price=398.53, time_in_force='2023-03-11 14:10:22')
Order Placed Successfully
Trade(symbol='FB', qty=2, price=398.53, side='BUY', timestamp='2023-03-11T19:10:22.815077', order_id='5e3c0e2b-2abb-4601-b414-d138683d4061', instmt='FB', trade_price=398.53, trade_qty=2, trade_side='BUY', trade_id='06a4e2ec-4551-435c-9313-7c356a2893dd')
Randomly generated order statement 44: Order(symbol='GOOG', qty=4, side='SELL', type='limit', price=1151.28, time_in_force='2023-03-11 14:10:22')
Order Placed Successfully
Trade(symbol='GOOG', qty=4, price=1151.28, side='SELL', timestamp='2023-03-11T19:10:22.819634', order_id='0985c104-4a03-

Order Placed Successfully
Trade(symbol='GOOG', qty=4, price=1143.77, side='SELL', timestamp='2023-03-11T19:10:23.017330', order_id='7f741120-9cd6-4ee4-a34e-400b2e1cbe92', instmt='GOOG', trade_price=1143.77, trade_qty=4, trade_side='SELL', trade_id='b15fd4ac-7bf1-4465-8237-bdd818489a69')
Randomly generated order statement 86: Order(symbol='TSLA', qty=8, side='BUY', type='limit', price=782.71, time_in_force='2023-03-11 14:10:23')
Order Placed Successfully
Trade(symbol='TSLA', qty=8, price=782.71, side='BUY', timestamp='2023-03-11T19:10:23.021559', order_id='a655bffd-744c-4673-a936-7c7182c378cb', instmt='TSLA', trade_price=782.71, trade_qty=8, trade_side='BUY', trade_id='71e87f12-9b93-4c8e-be40-8d70e9ac56f8')
Randomly generated order statement 87: Order(symbol='GOOG', qty=1, side='BUY', type='limit', price=1169.55, time_in_force='2023-03-11 14:10:23')
Order Placed Successfully
Trade(symbol='GOOG', qty=1, price=1169.55, side='BUY', timestamp='2023-03-11T19:10:23.025304', order_id='194f9875